Election de 2022 premier tour

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import missingno as msno
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

import os
os.listdir()

['election2017.ipynb',
 '.DS_Store',
 'enums',
 'utils',
 'models',
 'election2012.ipynb',
 'labo1.ipynb',
 '.gitignore',
 'scripts',
 'db',
 'labo2.ipynb',
 '.git',
 'main.py',
 'election2022.ipynb',
 'elections']

#### Importer la database

In [19]:
from db import SessionLocal
session = SessionLocal()

Import des datasets.

In [20]:
df = pd.read_excel('./elections/presidentielles-2012-1.xlsx')

Visualise toutes les colones

In [21]:
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

df.head(1000)

,Code du département,Libellé du département,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,Blancs et nuls,% BlNuls/Ins,% BlNuls/Vot,Exprimés,% Exp/Ins,% Exp/Vot,Sexe,Nom,Prénom,Voix,% Voix/Ins,% Voix/Exp,Sexe.1,Nom.1,Prénom.1,Voix.1,% Voix/Ins.1,% Voix/Exp.1,Sexe.2,Nom.2,Prénom.2,Voix.2,% Voix/Ins.2,% Voix/Exp.2,Sexe.3,Nom.3,Prénom.3,Voix.3,% Voix/Ins.3,% Voix/Exp.3,Sexe.4,Nom.4,Prénom.4,Voix.4,% Voix/Ins.4,% Voix/Exp.4,Sexe.5,Nom.5,Prénom.5,Voix.5,% Voix/Ins.5,% Voix/Exp.5,Sexe.6,Nom.6,Prénom.6,Voix.6,% Voix/Ins.6,% Voix/Exp.6,Sexe.7,Nom.7,Prénom.7,Voix.7,% Voix/Ins.7,% Voix/Exp.7,Sexe.8,Nom.8,Prénom.8,Voix.8,% Voix/Ins.8,% Voix/Exp.8,Sexe.9,Nom.9,Prénom.9,Voix.9,% Voix/Ins.9,% Voix/Exp.9
0,1,AIN,393808,65996,16.76,327812,83.24,6453,1.64,1.97,321359,81.60,98.03,F,JOLY,Eva,7268,1.85,2.26,F,LE PEN,Marine,66540,16.90,20.71,M,SARKOZY,Nicolas,97722,24.81,30.41,M,MÉLENCHON,Jean-Luc,30898,7.85,9.61,M,POUTOU,Philippe,3323,0.84,1.03,F,ARTHAUD,Nathalie,1794,0.46,0.56,M,CHEMINADE,Jacques,860,0.22,0.27,M,BAYROU,François,32650,8.29,10.16,M,DUPONT-AIGNAN,Nicolas,7208,1.83,2.24,M,HOLLANDE,François,73096,18.56,22.75
1,2,AISNE,376068,72928,19.39,303140,80.61,5196,1.38,1.71,297944,79.23,98.29,F,JOLY,Eva,3455,0.92,1.16,F,LE PEN,Marine,78452,20.86,26.33,M,SARKOZY,Nicolas,72090,19.17,24.20,M,MÉLENCHON,Jean-Luc,30360,8.07,10.19,M,POUTOU,Philippe,3860,1.03,1.30,F,ARTHAUD,Nathalie,2490,0.66,0.84,M,CHEMINADE,Jacques,738,0.20,0.25,M,BAYROU,François,19895,5.29,6.68,M,DUPONT-AIGNAN,Nicolas,5853,1.56,1.96,M,HOLLANDE,François,80751,21.47,27.10
2,3,ALLIER,256275,45266,17.66,211009,82.34,5059,1.97,2.40,205950,80.36,97.60,F,JOLY,Eva,3232,1.26,1.57,F,LE PEN,Marine,37736,14.72,18.32,M,SARKOZY,Nicolas,49477,19.31,24.02,M,MÉLENCHON,Jean-Luc,27969,10.91,13.58,M,POUTOU,Philippe,2584,1.01,1.25,F,ARTHAUD,Nathalie,1482,0.58,0.72,M,CHEMINADE,Jacques,457,0.18,0.22,M,BAYROU,François,17814,6.95,8.65,M,DUPONT-AIGNAN,Nicolas,4068,1.59,1.98,M,HOLLANDE,François,61131,23.85,29.68
3,4,ALPES DE HAUTE PROVENCE,123933,21034,16.97,102899,83.03,2111,1.70,2.05,100788,81.32,97.95,F,JOLY,Eva,2933,2.37,2.91,F,LE PEN,Marine,20875,16.84,20.71,M,SARKOZY,Nicolas,25668,20.71,25.47,M,MÉLENCHON,Jean-Luc,15269,12.32,15.15,M,POUTOU,Philippe,1394,1.12,1.38,F,ARTHAUD,Nathalie,487,0.39,0.48,M,CHEMINADE,Jacques,283,0.23,0.28,M,BAYROU,François,7483,6.04,7.42,M,DUPONT-AIGNAN,Nicolas,1845,1.49,1.83,M,HOLLANDE,François,24551,19.81,24.36
4,5,HAUTES ALPES,106865,18246,17.07,88619,82.93,1842,1.72,2.08,86777,81.20,97.92,F,JOLY,Eva,3147,2.94,3.63,F,LE PEN,Marine,15359,14.37,17.70,M,SARKOZY,Nicolas,22655,21.20,26.11,M,MÉLENCHON,Jean-Luc,12175,11.39,14.03,M,POUTOU,Philippe,1152,1.08,1.33,F,ARTHAUD,Nathalie,488,0.46,0.56,M,CHEMINADE,Jacques,212,0.20,0.24,M,BAYROU,François,8559,8.01,9.86,M,DUPONT-AIGNAN,Nicolas,1782,1.67,2.05,M,HOLLANDE,François,21248,19.88,24.49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,ZP,POLYNESIE FRANCAISE,186642,94542,50.65,92100,49.35,2281,1.22,2.48,89819,48.12,97.52,F,JOLY,Eva,3392,1.82,3.78,F,LE PEN,Marine,5151,2.76,5.73,M,SARKOZY,Nicolas,40611,21.76,45.21,M,MÉLENCHON,Jean-Luc,2492,1.34,2.77,M,POUTOU,Philippe,532,0.29,0.59,F,ARTHAUD,Nathalie,510,0.27,0.57,M,CHEMINADE,Jacques,378,0.20,0.42,M,BAYROU,François,5139,2.75,5.72,M,DUPONT-AIGNAN,Nicolas,2484,1.33,2.77,M,HOLLANDE,François,29130,15.61,32.43
103,ZS,SAINT PIERRE ET MIQUELON,4923,2193,44.55,2730,55.45,99,2.01,3.63,2631,53.44,96.37,F,JOLY,Eva,43,0.87,1.63,F,LE PEN,Marine,416,8.45,15.81,M,SARKOZY,Nicolas,488,9.91,18.55,M,MÉLENCHON,Jean-Luc,399,8.10,15.17,M,POUTOU,Philippe,103,2.09,3.91,F,ARTHAUD,Nathalie,23,0.47,0.87,M,CHEMINADE,Jacques,13,0.26,0.49,M,BAYROU,François,194,3.94,7.37,M,DUPONT-AIGNAN,Nicolas,64,1.30,2.43,M,HOLLANDE,François,888,18.04,33.75
104,ZW,WALLIS-ET-FUTUNA,8940

In [22]:
#df.drop(index=107, inplace=True)

supprimer la colonne Etat saisie d’un DataFrame pandas.

In [23]:
df = df.drop(columns=["Etat saisie"], errors="ignore")

In [24]:
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

df.head(1000)

,Code du département,Libellé du département,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,Blancs et nuls,% BlNuls/Ins,% BlNuls/Vot,Exprimés,% Exp/Ins,% Exp/Vot,Sexe,Nom,Prénom,Voix,% Voix/Ins,% Voix/Exp,Sexe.1,Nom.1,Prénom.1,Voix.1,% Voix/Ins.1,% Voix/Exp.1,Sexe.2,Nom.2,Prénom.2,Voix.2,% Voix/Ins.2,% Voix/Exp.2,Sexe.3,Nom.3,Prénom.3,Voix.3,% Voix/Ins.3,% Voix/Exp.3,Sexe.4,Nom.4,Prénom.4,Voix.4,% Voix/Ins.4,% Voix/Exp.4,Sexe.5,Nom.5,Prénom.5,Voix.5,% Voix/Ins.5,% Voix/Exp.5,Sexe.6,Nom.6,Prénom.6,Voix.6,% Voix/Ins.6,% Voix/Exp.6,Sexe.7,Nom.7,Prénom.7,Voix.7,% Voix/Ins.7,% Voix/Exp.7,Sexe.8,Nom.8,Prénom.8,Voix.8,% Voix/Ins.8,% Voix/Exp.8,Sexe.9,Nom.9,Prénom.9,Voix.9,% Voix/Ins.9,% Voix/Exp.9
0,1,AIN,393808,65996,16.76,327812,83.24,6453,1.64,1.97,321359,81.60,98.03,F,JOLY,Eva,7268,1.85,2.26,F,LE PEN,Marine,66540,16.90,20.71,M,SARKOZY,Nicolas,97722,24.81,30.41,M,MÉLENCHON,Jean-Luc,30898,7.85,9.61,M,POUTOU,Philippe,3323,0.84,1.03,F,ARTHAUD,Nathalie,1794,0.46,0.56,M,CHEMINADE,Jacques,860,0.22,0.27,M,BAYROU,François,32650,8.29,10.16,M,DUPONT-AIGNAN,Nicolas,7208,1.83,2.24,M,HOLLANDE,François,73096,18.56,22.75
1,2,AISNE,376068,72928,19.39,303140,80.61,5196,1.38,1.71,297944,79.23,98.29,F,JOLY,Eva,3455,0.92,1.16,F,LE PEN,Marine,78452,20.86,26.33,M,SARKOZY,Nicolas,72090,19.17,24.20,M,MÉLENCHON,Jean-Luc,30360,8.07,10.19,M,POUTOU,Philippe,3860,1.03,1.30,F,ARTHAUD,Nathalie,2490,0.66,0.84,M,CHEMINADE,Jacques,738,0.20,0.25,M,BAYROU,François,19895,5.29,6.68,M,DUPONT-AIGNAN,Nicolas,5853,1.56,1.96,M,HOLLANDE,François,80751,21.47,27.10
2,3,ALLIER,256275,45266,17.66,211009,82.34,5059,1.97,2.40,205950,80.36,97.60,F,JOLY,Eva,3232,1.26,1.57,F,LE PEN,Marine,37736,14.72,18.32,M,SARKOZY,Nicolas,49477,19.31,24.02,M,MÉLENCHON,Jean-Luc,27969,10.91,13.58,M,POUTOU,Philippe,2584,1.01,1.25,F,ARTHAUD,Nathalie,1482,0.58,0.72,M,CHEMINADE,Jacques,457,0.18,0.22,M,BAYROU,François,17814,6.95,8.65,M,DUPONT-AIGNAN,Nicolas,4068,1.59,1.98,M,HOLLANDE,François,61131,23.85,29.68
3,4,ALPES DE HAUTE PROVENCE,123933,21034,16.97,102899,83.03,2111,1.70,2.05,100788,81.32,97.95,F,JOLY,Eva,2933,2.37,2.91,F,LE PEN,Marine,20875,16.84,20.71,M,SARKOZY,Nicolas,25668,20.71,25.47,M,MÉLENCHON,Jean-Luc,15269,12.32,15.15,M,POUTOU,Philippe,1394,1.12,1.38,F,ARTHAUD,Nathalie,487,0.39,0.48,M,CHEMINADE,Jacques,283,0.23,0.28,M,BAYROU,François,7483,6.04,7.42,M,DUPONT-AIGNAN,Nicolas,1845,1.49,1.83,M,HOLLANDE,François,24551,19.81,24.36
4,5,HAUTES ALPES,106865,18246,17.07,88619,82.93,1842,1.72,2.08,86777,81.20,97.92,F,JOLY,Eva,3147,2.94,3.63,F,LE PEN,Marine,15359,14.37,17.70,M,SARKOZY,Nicolas,22655,21.20,26.11,M,MÉLENCHON,Jean-Luc,12175,11.39,14.03,M,POUTOU,Philippe,1152,1.08,1.33,F,ARTHAUD,Nathalie,488,0.46,0.56,M,CHEMINADE,Jacques,212,0.20,0.24,M,BAYROU,François,8559,8.01,9.86,M,DUPONT-AIGNAN,Nicolas,1782,1.67,2.05,M,HOLLANDE,François,21248,19.88,24.49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,ZP,POLYNESIE FRANCAISE,186642,94542,50.65,92100,49.35,2281,1.22,2.48,89819,48.12,97.52,F,JOLY,Eva,3392,1.82,3.78,F,LE PEN,Marine,5151,2.76,5.73,M,SARKOZY,Nicolas,40611,21.76,45.21,M,MÉLENCHON,Jean-Luc,2492,1.34,2.77,M,POUTOU,Philippe,532,0.29,0.59,F,ARTHAUD,Nathalie,510,0.27,0.57,M,CHEMINADE,Jacques,378,0.20,0.42,M,BAYROU,François,5139,2.75,5.72,M,DUPONT-AIGNAN,Nicolas,2484,1.33,2.77,M,HOLLANDE,François,29130,15.61,32.43
103,ZS,SAINT PIERRE ET MIQUELON,4923,2193,44.55,2730,55.45,99,2.01,3.63,2631,53.44,96.37,F,JOLY,Eva,43,0.87,1.63,F,LE PEN,Marine,416,8.45,15.81,M,SARKOZY,Nicolas,488,9.91,18.55,M,MÉLENCHON,Jean-Luc,399,8.10,15.17,M,POUTOU,Philippe,103,2.09,3.91,F,ARTHAUD,Nathalie,23,0.47,0.87,M,CHEMINADE,Jacques,13,0.26,0.49,M,BAYROU,François,194,3.94,7.37,M,DUPONT-AIGNAN,Nicolas,64,1.30,2.43,M,HOLLANDE,François,888,18.04,33.75
104,ZW,WALLIS-ET-FUTUNA,8940

In [25]:
df.shape

(107, 73)

Recuperer les information généraux sur les élections départemental

In [26]:
df_infos_general = df.iloc[:, :13]
df_infos_general.head()

,Code du département,Libellé du département,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,Blancs et nuls,% BlNuls/Ins,% BlNuls/Vot,Exprimés,% Exp/Ins,% Exp/Vot
0,1,AIN,393808,65996,16.76,327812,83.24,6453,1.64,1.97,321359,81.60,98.03
1,2,AISNE,376068,72928,19.39,303140,80.61,5196,1.38,1.71,297944,79.23,98.29
2,3,ALLIER,256275,45266,17.66,211009,82.34,5059,1.97,2.40,205950,80.36,97.60
3,4,ALPES DE HAUTE PROVENCE,123933,21034,16.97,102899,83.03,2111,1.70,2.05,100788,81.32,97.95
4,5,HAUTES ALPES,106865,18246,17.07,88619,82.93,1842,1.72,2.08,86777,81.20,97.92


Renommer les colonnes des information généraux sur les élections départemental

In [27]:
df_infos_general.columns = [
    "code_dept", "nom_dept", "nb_inscrits", "nb_abstentions",
    "pct_abstentions", "nb_votants", "pct_votants", "nb_blancs_nuls",
    "pct_blancs_nuls_inscrits", "pct_blancs_nuls_votants", "nb_exprimes",
    "pct_exprimes_inscrits", "pct_exprimes_votants"
]
df_infos_general.head()

,code_dept,nom_dept,nb_inscrits,nb_abstentions,pct_abstentions,nb_votants,pct_votants,nb_blancs_nuls,pct_blancs_nuls_inscrits,pct_blancs_nuls_votants,nb_exprimes,pct_exprimes_inscrits,pct_exprimes_votants
0,1,AIN,393808,65996,16.76,327812,83.24,6453,1.64,1.97,321359,81.60,98.03
1,2,AISNE,376068,72928,19.39,303140,80.61,5196,1.38,1.71,297944,79.23,98.29
2,3,ALLIER,256275,45266,17.66,211009,82.34,5059,1.97,2.40,205950,80.36,97.60
3,4,ALPES DE HAUTE PROVENCE,123933,21034,16.97,102899,83.03,2111,1.70,2.05,100788,81.32,97.95
4,5,HAUTES ALPES,106865,18246,17.07,88619,82.93,1842,1.72,2.08,86777,81.20,97.92


In [28]:
df_infos_general.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   code_dept                 107 non-null    object 
 1   nom_dept                  107 non-null    object 
 2   nb_inscrits               107 non-null    int64  
 3   nb_abstentions            107 non-null    int64  
 4   pct_abstentions           107 non-null    float64
 5   nb_votants                107 non-null    int64  
 6   pct_votants               107 non-null    float64
 7   nb_blancs_nuls            107 non-null    int64  
 8   pct_blancs_nuls_inscrits  107 non-null    float64
 9   pct_blancs_nuls_votants   107 non-null    float64
 10  nb_exprimes               107 non-null    int64  
 11  pct_exprimes_inscrits     107 non-null    float64
 12  pct_exprimes_votants      107 non-null    float64
dtypes: float64(6), int64(5), object(2)
memory usage: 11.0+ KB


Suppression des données redondantes, risque de multicolinéarité en ML

In [29]:
colonnes_base = [
    "code_dept", "nom_dept",
    "nb_inscrits", "nb_abstentions", "nb_votants", 
    "nb_blancs_nuls",
]

df_infos_general_base = df_infos_general[colonnes_base]
df_infos_general_base.head()

,code_dept,nom_dept,nb_inscrits,nb_abstentions,nb_votants,nb_blancs_nuls
0,1,AIN,393808,65996,327812,6453
1,2,AISNE,376068,72928,303140,5196
2,3,ALLIER,256275,45266,211009,5059
3,4,ALPES DE HAUTE PROVENCE,123933,21034,102899,2111
4,5,HAUTES ALPES,106865,18246,88619,1842


In [30]:
df_infos_general_base.shape

(107, 6)

In [31]:
df_departement = df_infos_general_base.iloc[:, :2]

pd.set_option('display.max_rows', None)      # Affiche toutes les lignes
pd.set_option('display.max_columns', None)   # Affiche toutes les colonnes
pd.set_option('display.width', None)         # Évite le retour à la ligne automatique
pd.set_option('display.max_colwidth', None)  # Affiche tout le contenu des cellules

print(df_departement)
#df_departement.head()

    code_dept                       nom_dept
0           1                            AIN
1           2                          AISNE
2           3                         ALLIER
3           4        ALPES DE HAUTE PROVENCE
4           5                   HAUTES ALPES
5           6                ALPES MARITIMES
6           7                        ARDECHE
7           8                       ARDENNES
8           9                         ARIEGE
9          10                           AUBE
10         11                           AUDE
11         12                        AVEYRON
12         13               BOUCHES DU RHONE
13         14                       CALVADOS
14         15                         CANTAL
15         16                       CHARENTE
16         17              CHARENTE MARITIME
17         18                           CHER
18         19                        CORREZE
19         2A                      CORSE SUD
20         2B                    HAUTE CORSE
21        

In [32]:
df_stat_elections = df_infos_general_base.drop(columns=["nom_dept"])
df_stat_elections.head()

,code_dept,nb_inscrits,nb_abstentions,nb_votants,nb_blancs_nuls
0,1,393808,65996,327812,6453
1,2,376068,72928,303140,5196
2,3,256275,45266,211009,5059
3,4,123933,21034,102899,2111
4,5,106865,18246,88619,1842


Tratification pour consever les proportions

In [33]:
import pandas as pd

# Colonnes globales que l'on veut garder
cols_globales = ['Code du département', 'Libellé du département']

# Colonnes répétitives candidats
df_candidates = df.iloc[:, 13:]  # après % Exp/Vot
cols_per_candidate = 6
num_candidates = df_candidates.shape[1] // cols_per_candidate

dfs_list = []

for i in range(num_candidates):
    start_col = i * cols_per_candidate
    end_col = start_col + cols_per_candidate
    df_cand = df_candidates.iloc[:, start_col:end_col].copy()
    df_cand.columns = ['Sexe', 'Nom', 'Prenom', 'Voix', '% Voix/Ins', '% Voix/Exp']
    
    # Ajouter les informations de département
    df_cand = pd.concat([df[cols_globales], df_cand], axis=1)
    
    dfs_list.append(df_cand)

# Combiner tous les candidats
df_simple = pd.concat(dfs_list, ignore_index=True)

# Colonnes à garder
cols_to_keep = ['Code du département', 'Libellé du département', 'Sexe', 'Nom', 'Prenom', 'Voix']

# Nouveau dataframe simplifié
df_candidat_resultat = df_simple[cols_to_keep]

# Afficher toutes les lignes
pd.set_option('display.max_rows', 10)  # ou un nombre supérieur à 100
pd.set_option('display.max_columns', None)  # pour voir toutes les colonnes
pd.set_option('display.width', None)        # pour éviter la coupure à la largeur de l'écran



#df_candidat_resultat = df_candidat_resultat.drop(index=215)
# Affichage des 5 premières lignes
df_candidat_resultat.head(1000)


,Code du département,Libellé du département,Sexe,Nom,Prenom,Voix
0,1,AIN,F,JOLY,Eva,7268
1,2,AISNE,F,JOLY,Eva,3455
2,3,ALLIER,F,JOLY,Eva,3232
3,4,ALPES DE HAUTE PROVENCE,F,JOLY,Eva,2933
4,5,HAUTES ALPES,F,JOLY,Eva,3147
...,...,...,...,...,...,...
995,32,GERS,M,HOLLANDE,François,38446
996,33,GIRONDE,M,HOLLANDE,François,260043
997,34,HERAULT,M,HOLLANDE,François,160931
998,35,ILLE ET VILAINE,M,HOLLANDE,François,183935


In [34]:
from datetime import date

from utils.election_importer import ElectionImporter
from enums.type_election import TypeElection

# =====================================================
# Instanciation
# =====================================================
importer = ElectionImporter()

# =====================================================
# Élection
# =====================================================
election = importer.get_or_create_election(
    election_date=date(2012, 4, 23),
    type_election=TypeElection.PRESIDENTIELLE,
    tour=1
)

# =====================================================
# Départements
# =====================================================
#df_departement = edf.df_departement
if df_departement is not None and not df_departement.empty:
    importer.import_departements(df_departement)

# =====================================================
# Statistiques par département
# =====================================================
#df_stat_elections = edf.df_stat_elections
if df_stat_elections is not None and not df_stat_elections.empty:
    importer.import_stats(df_stat_elections, election.id)

# =====================================================
# Candidats & résultats
# =====================================================
#df_candidat_resultat = edf.df_candidat_resultat
if df_candidat_resultat is not None and not df_candidat_resultat.empty:
    importer.import_candidats_resultats(df_candidat_resultat, election.id)

print("✅ Import élection 2017 – Tour 1 terminé avec succès")


2025-12-26 11:44:45,609 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-26 11:44:45,617 INFO sqlalchemy.engine.Engine SELECT elections.id AS elections_id, elections.date AS elections_date, elections.type_election AS elections_type_election, elections.tour AS elections_tour 
FROM elections 
WHERE elections.date = %(date_1)s AND elections.type_election = %(type_election_1)s AND elections.tour = %(tour_1)s 
 LIMIT %(param_1)s
2025-12-26 11:44:45,619 INFO sqlalchemy.engine.Engine [cached since 3182s ago] {'date_1': datetime.date(2012, 4, 23), 'type_election_1': 'PRESIDENTIELLE', 'tour_1': 1, 'param_1': 1}
2025-12-26 11:44:45,639 INFO sqlalchemy.engine.Engine INSERT INTO elections (date, type_election, tour) VALUES (%(date)s, %(type_election)s, %(tour)s) RETURNING elections.id
2025-12-26 11:44:45,640 INFO sqlalchemy.engine.Engine [cached since 3182s ago] {'date': datetime.date(2012, 4, 23), 'type_election': 'PRESIDENTIELLE', 'tour': 1}
2025-12-26 11:44:45,644 INFO sqlalchemy.engine.E